In [6]:
import pandas as pd
from utils.gcs import list_blobs, list_blobs_with_prefix, get_text_file, get_blob, upload_many_blobs_with_transfer_manager
from glob import glob
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
import pickle

# Load Source Data

The train and test datsets have class labels representing the yoga pose and a filepath which provides urls linking to images for the particular pose.

In [7]:
train = pd.read_csv('gs://ai-yoga-trainer-data/source/yoga_train.txt', header=None, names=['filepath', 'class_6', 'class_20', 'class_82'])
test = pd.read_csv('gs://ai-yoga-trainer-data/source/yoga_test.txt', header=None, names=['filepath', 'class_6', 'class_20', 'class_82'])

In [8]:
train.head()

,filepath,class_6,class_20,class_82
0,Akarna_Dhanurasana/64.jpg,1,8,0
1,Akarna_Dhanurasana/229.jpg,1,8,0
2,Akarna_Dhanurasana/128.jpg,1,8,0
3,Akarna_Dhanurasana/145.jpg,1,8,0
4,Akarna_Dhanurasana/47.jpg,1,8,0


# Generate Mapper for Filename to Image URL

Here, we will create a map between filenames and URLs. This map is indirectly provided in the source data, as there is a textfile for each pose containing the URLs linking to the images for that particular pose. We will iterate through all textfiles to create a global map. 

In [9]:
dfs = []
for blob in list_blobs_with_prefix('ai-yoga-trainer-data', prefix='source/yoga_dataset_links/'):
    df = pd.read_csv(f'gs://ai-yoga-trainer-data/{blob.name}', delimiter='\t', header=None, names=['filepath', 'image_url'])
    df['pose_name'] = blob.name.split('/')[-1].split('.txt')[0]
    dfs.append(df)
filepath_img_url_map = pd.concat(dfs)
filepath_img_url_map.head()

,filepath,image_url,pose_name
0,Akarna_Dhanurasana/680.jpg,http://allyogapositions.com/wp-content/uploads...,Akarna_Dhanurasana
1,Akarna_Dhanurasana/543.jpg,https://image.slidesharecdn.com/basicyogapostu...,Akarna_Dhanurasana
2,Akarna_Dhanurasana/512.jpg,https://image.slidesharecdn.com/basicyogapostu...,Akarna_Dhanurasana
3,Akarna_Dhanurasana/478.jpg,http://nicholchaseyoga.com/wp-content/uploads/...,Akarna_Dhanurasana
4,Akarna_Dhanurasana/434.jpg,http://3.bp.blogspot.com/_SUO8HYzS2nw/SUPSs71K...,Akarna_Dhanurasana


Here, we will merge the original train data with the map to append the class labels.

In [10]:
train = train.merge(filepath_img_url_map, left_on='filepath', right_on='filepath')
test = test.merge(filepath_img_url_map, left_on='filepath', right_on='filepath')

In [11]:
train['img_num'] = train['filepath'].str.split('/').map(lambda x: x[1])
train['pose_name'] = train['pose_name'].str.rstrip('_')
test['img_num'] = test['filepath'].str.split('/').map(lambda x: x[1])
test['pose_name'] = test['pose_name'].str.rstrip('_')


In [12]:
train.head()

,filepath,class_6,class_20,class_82,image_url,pose_name,img_num
0,Akarna_Dhanurasana/64.jpg,1,8,0,http://st1.thehealthsite.com/wp-content/upload...,Akarna_Dhanurasana,64.jpg
1,Akarna_Dhanurasana/229.jpg,1,8,0,http://www.yoga-hb.jp/wp-content/uploads/2015/...,Akarna_Dhanurasana,229.jpg
2,Akarna_Dhanurasana/128.jpg,1,8,0,http://www.jaisiyaram.com/photopost/watermark....,Akarna_Dhanurasana,128.jpg
3,Akarna_Dhanurasana/145.jpg,1,8,0,http://mugup.info/appimg/Yoga/Akarna%20Dhanura...,Akarna_Dhanurasana,145.jpg
4,Akarna_Dhanurasana/47.jpg,1,8,0,https://biocentroshantala.es/wp-content/upload...,Akarna_Dhanurasana,47.jpg


# Get Images

We will use the requests library to get the images for each URL.

In [58]:
def get_image(url):
    try:
        r = requests.get(url, stream=True, verify=False)
        img_bytes = BytesIO(r.content)
        im = Image.open(img_bytes)
    except Exception:
        return np.nan
    return im

In [ ]:
train_imgs = []
for url in tqdm(train['image_url']):
    train_imgs.append(get_image(url))
train['img'] = train_imgs

In [ ]:
test_imgs = []
for url in tqdm(test['image_url']):
    test_imgs.append(get_image(url))
test['img'] = test_imgs

# Load Kaggle Data

After running the above loops, it seems many links are broken. Luckily, someone already processed these text files to extract the images and posted them on [Kaggle](https://www.kaggle.com/datasets/akashrayhan/yoga-82?resource=download). Although the classnames are provided, the creators of the dataset boast that it has a hierachical structure. The Kaggle version of the dataset does not contain hierarchical labels, so will try to merge the train data above to the Kaggle data.

In [13]:
from glob import glob

In [14]:
def load_imgs(path):
    imgs = {}
    for path_to_pose in glob(path):
        for path_to_img in glob(f'{path_to_pose}/*.jpg'):
            im = cv2.imread(path_to_img)
            im_name = path_to_img.rsplit('/')[-1]
            imgs[im_name] = im
    return imgs

def process_imgs(imgs):
    imgs = pd.DataFrame(zip(*[imgs.keys(), imgs.values()]), columns=['image_name', 'image'])
    imgs['pose_name'] = imgs['image_name'].str.split('image').map(lambda x: x[0])
    imgs['pose_name'] = imgs['pose_name'].map(lambda x: x.rstrip('_'))
    imgs['image_num'] = imgs['image_name'].str.split('image').map(lambda x: x[1])
    imgs['image_num'] = imgs['image_num'].map(lambda x: x.lstrip('_'))
    imgs['pose_name'] = imgs['pose_name'].str.replace('Bharadvajas_Twist_pose_or_Bharadvajasana_I', "Bharadvaja's_Twist_pose_or_Bharadvajasana_I")

    return imgs



In [15]:
train_imgs = load_imgs('./data/kaggle/train/*')
train_imgs = process_imgs(train_imgs)
train_imgs.head()

libpng warning: iCCP: profile 'ICC Profile': 'RGB ': RGB color space not permitted on grayscale PNG
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: cHRM: inconsistent chromaticities
Corrupt JPEG data: 35 extraneous bytes before marker 0xd9
libpng warning: iCCP: known incorrect sRGB profile
Corrupt JPEG data: 2932 extraneous bytes before marker 0xed
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


,image_name,image,pose_name,image_num
0,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,16.jpg
1,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[140, 160, 178], [140, 160, 178], [141, 161,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,17.jpg
2,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[159, 164, 167], [160, 165, 168], [161, 166,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,15.jpg
3,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[251, 253, 253], [251, 253, 253], [251, 253,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,29.jpg
4,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,28.jpg


In [16]:
test_imgs = load_imgs('./data/kaggle/test/*')
test_imgs = process_imgs(test_imgs)
test_imgs.head()

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


,image_name,image,pose_name,image_num
0,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[223, 242, 245], [223, 242, 245], [223, 242,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,173.jpg
1,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[255, 255, 255], [254, 254, 254], [246, 246,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,175.jpg
2,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[245, 245, 245], [245, 245, 245], [245, 245,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,177.jpg
3,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,188.jpg
4,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[161, 156, 147], [162, 157, 148], [164, 159,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,189.jpg


In [17]:
valid_imgs = load_imgs('./data/kaggle/valid/*')
valid_imgs = process_imgs(valid_imgs)
valid_imgs.head()

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


,image_name,image,pose_name,image_num
0,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,166.jpg
1,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,167.jpg
2,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,159.jpg
3,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,171.jpg
4,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,165.jpg


Let's undo the train test split and concatenate all the data together.

In [18]:
all_imgs = pd.concat([train_imgs, valid_imgs, test_imgs]).reset_index(drop=True)
all_imgs.head()

,image_name,image,pose_name,image_num
0,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,16.jpg
1,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[140, 160, 178], [140, 160, 178], [141, 161,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,17.jpg
2,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[159, 164, 167], [160, 165, 168], [161, 166,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,15.jpg
3,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[251, 253, 253], [251, 253, 253], [251, 253,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,29.jpg
4,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,28.jpg


Let's first split the pose name into its main name and alternate name.

In [19]:
all_imgs['pose_name_alt'] = all_imgs['pose_name'].map(lambda x: x.split('_or_')[1] if '_or_' in x else np.nan)
all_imgs.head()

,image_name,image,pose_name,image_num,pose_name_alt
0,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,16.jpg,Janu_Sirsasana
1,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[140, 160, 178], [140, 160, 178], [141, 161,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,17.jpg,Janu_Sirsasana
2,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[159, 164, 167], [160, 165, 168], [161, 166,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,15.jpg,Janu_Sirsasana
3,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[251, 253, 253], [251, 253, 253], [251, 253,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,29.jpg,Janu_Sirsasana
4,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasana,28.jpg,Janu_Sirsasana


In [20]:
all_imgs['pose_name'] = all_imgs['pose_name'].map(lambda x: x.split('_or_')[0] if '_or_' in x else x)
all_imgs.head()

,image_name,image,pose_name,image_num,pose_name_alt
0,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Head-to-Knee_Forward_Bend_pose,16.jpg,Janu_Sirsasana
1,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[140, 160, 178], [140, 160, 178], [141, 161,...",Head-to-Knee_Forward_Bend_pose,17.jpg,Janu_Sirsasana
2,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[159, 164, 167], [160, 165, 168], [161, 166,...",Head-to-Knee_Forward_Bend_pose,15.jpg,Janu_Sirsasana
3,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[251, 253, 253], [251, 253, 253], [251, 253,...",Head-to-Knee_Forward_Bend_pose,29.jpg,Janu_Sirsasana
4,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Head-to-Knee_Forward_Bend_pose,28.jpg,Janu_Sirsasana


In [21]:
all_imgs['pose_name'] = all_imgs['pose_name'].map(lambda x: ' '.join(x.split('_')))
all_imgs['pose_name_alt'] = all_imgs['pose_name_alt'].map(lambda x: ' '.join(x.split('_')) if type(x) == str else x)
all_imgs.head()

,image_name,image,pose_name,image_num,pose_name_alt
0,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Head-to-Knee Forward Bend pose,16.jpg,Janu Sirsasana
1,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[140, 160, 178], [140, 160, 178], [141, 161,...",Head-to-Knee Forward Bend pose,17.jpg,Janu Sirsasana
2,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[159, 164, 167], [160, 165, 168], [161, 166,...",Head-to-Knee Forward Bend pose,15.jpg,Janu Sirsasana
3,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[251, 253, 253], [251, 253, 253], [251, 253,...",Head-to-Knee Forward Bend pose,29.jpg,Janu Sirsasana
4,Head-to-Knee_Forward_Bend_pose_or_Janu_Sirsasa...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Head-to-Knee Forward Bend pose,28.jpg,Janu Sirsasana


In [22]:
all_imgs['pose_name'] = all_imgs['pose_name'].str.title()
all_imgs['pose_name_alt'] = all_imgs['pose_name_alt'].str.title()

In [67]:
with pd.option_context('display.max_rows', None):
    display(all_imgs[['pose_name', 'pose_name_alt']].drop_duplicates())

,pose_name,pose_name_alt
0,Head-To-Knee Forward Bend Pose,Janu Sirsasana
129,Dolphin Pose,Ardha Pincha Mayurasana
171,Shoulder-Pressing Pose,Bhujapidasana
241,Corpse Pose,Savasana
430,Bridge Pose,Setu Bandha Sarvangasana
623,Standing Big Toe Hold Pose,Utthita Padangusthasana
732,Legs-Up-The-Wall Pose,Viparita Karani
892,Standing Forward Bend Pose,Uttanasana
1166,Warrior Ii Pose,Virabhadrasana Ii
1352,Fish Pose,Matsyasana


Let's fix some formatting issues.

In [23]:
def replace_roman_numerals(pose):
    if type(pose) == str:
        pose = pose.replace('Iii', '3')
        pose = pose.replace('Ii', '2')
        pose = pose.replace('I', '1')
        return pose
    return pose

In [24]:
all_imgs['pose_name'] = all_imgs['pose_name'].map(replace_roman_numerals)
all_imgs['pose_name_alt'] = all_imgs['pose_name_alt'].map(replace_roman_numerals)

In [25]:
all_imgs['pose_name'] = all_imgs['pose_name'].map(lambda x: 'Reverse Warrior Pose' if x == 'Viparita Virabhadrasana' else x)
all_imgs['pose_name_alt'] = all_imgs['pose_name_alt'].map(lambda x: 'Viparita Virabhadrasana' if x == 'Reverse Warrior Pose' else x)
all_imgs['pose_name'] = all_imgs['pose_name'].map(lambda x: "Bharadvaja's Twist Pose" if x == "Bharadvaja'S Twist Pose" else x)
all_imgs['pose_name'] = all_imgs['pose_name'].map(lambda x: 'Intense Side Stretch Pose' if x == '1ntense Side Stretch Pose' else x)

In [26]:
all_imgs['pose_name'] = all_imgs['pose_name'].map(lambda x: 'Shooting Bow Pose' if x == 'Akarna Dhanurasana' else x)
all_imgs['pose_name_alt'] = all_imgs.apply(lambda x: 'Akarna Dhanurasana' if x['pose_name'] == 'Shooting Bow Pose' else x['pose_name_alt'], axis=1)
all_imgs['pose_name'] = all_imgs['pose_name'].map(lambda x: 'One-Legged King Pigeon Pose' if x == 'Rajakapotasana' else x)
all_imgs['pose_name_alt'] = all_imgs.apply(lambda x: 'Rajakapotasana' if x['pose_name'] == 'One-Legged King Pigeon Pose' else x['pose_name_alt'], axis=1)
all_imgs['pose_name'] = all_imgs['pose_name'].map(lambda x: 'Reclining Bound Angle Pose' if x == 'Supta Baddha Konasana' else x)
all_imgs['pose_name_alt'] = all_imgs.apply(lambda x: 'Supta Baddha Konasana' if x['pose_name'] == 'Reclining Bound Angle Pose' else x['pose_name_alt'], axis=1)
all_imgs['pose_name'] = all_imgs['pose_name'].map(lambda x: 'Hero Pose' if x == 'Virasana' else x)
all_imgs['pose_name_alt'] = all_imgs.apply(lambda x: 'Virasana' if x['pose_name'] == 'Hero Pose' else x['pose_name_alt'], axis=1)
all_imgs['pose_name'] = all_imgs['pose_name'].map(lambda x: 'Reclining Hero Pose' if x == 'Supta Virasana Vajrasana' else x)
all_imgs['pose_name_alt'] = all_imgs.apply(lambda x: 'Supta Virasana' if x['pose_name'] == 'Reclining Hero Pose' else x['pose_name_alt'], axis=1)

In [27]:
with pd.option_context('display.max_rows', None):
    display(all_imgs[['pose_name', 'pose_name_alt']].drop_duplicates())

,pose_name,pose_name_alt
0,Head-To-Knee Forward Bend Pose,Janu Sirsasana
129,Dolphin Pose,Ardha Pincha Mayurasana
171,Shoulder-Pressing Pose,Bhujapidasana
241,Corpse Pose,Savasana
430,Bridge Pose,Setu Bandha Sarvangasana
623,Standing Big Toe Hold Pose,Utthita Padangusthasana
732,Legs-Up-The-Wall Pose,Viparita Karani
892,Standing Forward Bend Pose,Uttanasana
1166,Warrior 2 Pose,Virabhadrasana 2
1352,Fish Pose,Matsyasana


Now let's add the hierachical labels. The original [paper](https://arxiv.org/pdf/2004.10362.pdf) provides this map.

In [28]:
all_pose_names = [('Head-To-Knee Forward Bend Pose', 'Forward Bend', 'Sitting'),
 ('Dolphin Pose', 'Forward Bend', 'Standing'),
 ('Shoulder-Pressing Pose', 'Front', 'Balancing'),
 ('Corpse Pose', 'Up-Facing', 'Reclining'),
 ('Bridge Pose', 'Up-Facing', 'Wheel'),
 ('Standing Big Toe Hold Pose', 'Others', 'Standing'),
 ('Legs-Up-The-Wall Pose', 'Legs-Straight-Up', 'Inverted'),
 ('Standing Forward Bend Pose', 'Forward Bend', 'Standing'),
 ('Warrior 2 Pose', 'Others', 'Standing'),
 ('Fish Pose', 'Up-Facing', 'Reclining'),
 ('Camel Pose', 'Up-Facing', 'Wheel'),
 ('Split Pose', 'Split', 'Sitting'),
 ('Bow Pose', 'Others', 'Wheel'),
 ('Garland Pose', 'Legs-In-Front', 'Sitting'),
 ('Upward Facing Two-Foot Staff Pose', 'Up-Facing', 'Wheel'),
 ('Child Pose', 'Down-Facing', 'Reclining'),
 ('Chair Pose', 'Straight', 'Standing'),
 ('Cockerel Pose', 'Front', 'Balancing'),
 ('Warrior 1 Pose', 'Side Bend', 'Standing'),
 ('Lord Of The Dance Pose', 'Others', 'Standing'),
('Tree Pose', 'Straight', 'Standing'),
 ('Reverse Warrior Pose', 'Side Bend', 'Standing'),
 ('Locust Pose', 'Down-Facing', 'Reclining'),
 ('Half Lord Of The Fishes Pose', 'Legs-Behind', 'Sitting'),
 ('Revolved Head-To-Knee Pose', 'Forward Bend', 'Sitting'),
 ('Pigeon Pose', 'Up-Facing', 'Wheel'),
 ('Scorpion Pose', 'Legs-Bend', 'Inverted'),
 ('Bound Angle Pose', 'Legs-In-Front', 'Sitting'),
 ('Half Moon Pose', 'Side Bend', 'Standing'),
 ('Staff Pose', 'Legs-In-Front', 'Sitting'),
 ('Eight-Angle Pose', 'Side', 'Balancing'),
 ('Side Crane (Crow) Pose', 'Side', 'Balancing'),
 ('Seated Forward Bend Pose', 'Forward Bend', 'Sitting'),
 ('Extended Revolved Side Angle Pose', 'Side Bend', 'Standing'),
 ('Downward-Facing Dog Pose', 'Forward Bend', 'Standing'),
 ('Firefly Pose', 'Front', 'Balancing'),
 ('Wild Thing Pose', 'Up-Facing', 'Wheel'),
 ('Sitting Pose 1 (Normal)', 'Legs-In-Front', 'Sitting'),
 ('Feathered Peacock Pose', 'Legs-Straight-Up', 'Inverted'),
 ('Shooting Bow Pose', 'Twist', 'Sitting'),
 ('Four-Limbed Staff Pose', 'Plank Balance', 'Reclining'),
 ('Gate Pose', 'Side Bend', 'Standing'),
 ('Handstand Pose', 'Legs-Straight-Up', 'Inverted'),
('Standing Split Pose', 'Others', 'Standing'),
 ('Supported Headstand Pose', 'Legs-Straight-Up', 'Inverted'),
 ('Side-Reclining Leg Lift Pose', 'Side-Facing', 'Reclining'),
 ('Upward Bow (Wheel) Pose', 'Up-Facing', 'Wheel'),
 ('Frog Pose', 'Down-Facing', 'Reclining'),
 ('Plow Pose', 'Legs-Bend', 'Inverted'),
 ('Crane (Crow) Pose', 'Front', 'Balancing'),
 ('Eagle Pose', 'Straight', 'Standing'),
 ('Wide-Legged Forward Bend Pose', 'Forward Bend', 'Standing'),
 ('Low Lunge Pose', 'Side Bend', 'Standing'),
 ('Cat Cow Pose', 'Down-Facing', 'Wheel'),
 ('Supported Shoulderstand Pose', 'Legs-Staight-Up', 'Inverted'),
 ('Noose Pose', 'Legs-In-Front', 'Sitting'),
 ('Cow Face Pose', 'Legs-Behind', 'Sitting'),
 ('Reclining Hand-To-Big-Toe Pose', 'Up-Facing', 'Reclining'),
 ('Pose Dedicated To The Sage Koundinya', 'Side', 'Balancing'),
 ('One-Legged King Pigeon Pose', 'Twist', 'Sitting'),
 ('Dolphin Plank Pose', 'Plank Balance', 'Reclining'),
 ('Happy Baby Pose', 'Up-Facing', 'Reclining'),
 ('Reclining Bound Angle Pose', 'Up-Facing', 'Reclining'),
 ('Upward Plank Pose', 'Up-Facing', 'Wheel'),
 ('Peacock Pose', 'Plank Balance', 'Reclining'),
 ('Boat Pose', 'Others', 'Wheel'),
 ('Side Plank Pose', 'Side-Facing', 'Reclining'),
 ('Extended Puppy Pose', 'Down-Facing', 'Reclining'),
 ('Extended Revolved Triangle Pose', 'Side Bend', 'Standing'),
 ('Wind Relieving Pose', 'Up-Facing', 'Reclining'),
 ('Wide-Angle Seated Forward Bend Pose', 'Split', 'Sitting'),
 ('Yogic Sleep Pose', 'Up-Facing', 'Reclining'),
 ('Warrior 3 Pose', 'Others', 'Standing'),
 ('Heron Pose', 'Twist', 'Sitting'),
 ('Plank Pose', 'Plank Balance', 'Reclining'),
 ('Scale Pose', 'Front', 'Balancing'),
 ('Cobra Pose', 'Down-Facing', 'Reclining'),
 ('Intense Side Stretch Pose', 'Forward Bend', 'Standing'),
 ('Hero Pose', 'Legs-Behind', 'Sitting'),
 ("Bharadvaja's Twist Pose", 'Legs-Behind', 'Sitting'),
 ('Tortoise Pose', 'Forward Bend', 'Sitting'),
 ('Reclining Hero Pose', 'Up-Facing', 'Reclining')]
all_pose_names = pd.DataFrame(all_pose_names, columns=['pose_name_l3', 'pose_name_l2', 'pose_name_l1'])
all_pose_names.head()

,pose_name_l3,pose_name_l2,pose_name_l1
0,Head-To-Knee Forward Bend Pose,Forward Bend,Sitting
1,Dolphin Pose,Forward Bend,Standing
2,Shoulder-Pressing Pose,Front,Balancing
3,Corpse Pose,Up-Facing,Reclining
4,Bridge Pose,Up-Facing,Wheel


Now, let's merge these hierarchical classes to the full dataset.

In [29]:
all_imgs = all_imgs.merge(all_pose_names, how='outer', left_on='pose_name', right_on='pose_name_l3')
all_imgs.head()

,image_name,image,pose_name,image_num,pose_name_alt,pose_name_l3,pose_name_l2,pose_name_l1
0,Bharadvajas_Twist_pose_or_Bharadvajasana_I__im...,"[[[160, 168, 21], [160, 168, 21], [160, 168, 2...",Bharadvaja's Twist Pose,53.jpg,Bharadvajasana 1,Bharadvaja's Twist Pose,Legs-Behind,Sitting
1,Bharadvajas_Twist_pose_or_Bharadvajasana_I__im...,"[[[247, 248, 244], [248, 249, 245], [248, 249,...",Bharadvaja's Twist Pose,47.jpg,Bharadvajasana 1,Bharadvaja's Twist Pose,Legs-Behind,Sitting
2,Bharadvajas_Twist_pose_or_Bharadvajasana_I__im...,"[[[220, 220, 220], [220, 220, 220], [220, 220,...",Bharadvaja's Twist Pose,46.jpg,Bharadvajasana 1,Bharadvaja's Twist Pose,Legs-Behind,Sitting
3,Bharadvajas_Twist_pose_or_Bharadvajasana_I__im...,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Bharadvaja's Twist Pose,52.jpg,Bharadvajasana 1,Bharadvaja's Twist Pose,Legs-Behind,Sitting
4,Bharadvajas_Twist_pose_or_Bharadvajasana_I__im...,"[[[211, 214, 219], [213, 216, 221], [210, 213,...",Bharadvaja's Twist Pose,44.jpg,Bharadvajasana 1,Bharadvaja's Twist Pose,Legs-Behind,Sitting


In [30]:
all_imgs = all_imgs[['image', 'pose_name_l3', 'pose_name_alt', 'pose_name_l2', 'pose_name_l1']].rename(columns={'pose_name_alt': 'pose_name_alt_l3'})
all_imgs.head()

,image,pose_name_l3,pose_name_alt_l3,pose_name_l2,pose_name_l1
0,"[[[160, 168, 21], [160, 168, 21], [160, 168, 2...",Bharadvaja's Twist Pose,Bharadvajasana 1,Legs-Behind,Sitting
1,"[[[247, 248, 244], [248, 249, 245], [248, 249,...",Bharadvaja's Twist Pose,Bharadvajasana 1,Legs-Behind,Sitting
2,"[[[220, 220, 220], [220, 220, 220], [220, 220,...",Bharadvaja's Twist Pose,Bharadvajasana 1,Legs-Behind,Sitting
3,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",Bharadvaja's Twist Pose,Bharadvajasana 1,Legs-Behind,Sitting
4,"[[[211, 214, 219], [213, 216, 221], [210, 213,...",Bharadvaja's Twist Pose,Bharadvajasana 1,Legs-Behind,Sitting


In [25]:
all_imgs.memory_usage(index=True, deep=True).sum()

37115410011

Let's add other statistics to the dataframe.

In [32]:
all_imgs['image_shape'] = all_imgs['image'].map(lambda x: x.shape if type(x) == np.ndarray else np.nan)
all_imgs['channel_B_pixel_mean'] = all_imgs['image'].map(lambda x: x[:,:,0].flatten().mean() if type(x) == np.ndarray else x)
all_imgs['channel_G_pixel_mean'] = all_imgs['image'].map(lambda x: x[:,:,1].flatten().mean() if type(x) == np.ndarray else x)
all_imgs['channel_R_pixel_mean'] = all_imgs['image'].map(lambda x: x[:,:,2].flatten().mean() if type(x) == np.ndarray else x)
all_imgs['channel_B_pixel_std'] = all_imgs['image'].map(lambda x: x[:,:,0].flatten().std() if type(x) == np.ndarray else x)
all_imgs['channel_G_pixel_std'] = all_imgs['image'].map(lambda x: x[:,:,1].flatten().std() if type(x) == np.ndarray else x)
all_imgs['channel_R_pixel_std'] = all_imgs['image'].map(lambda x: x[:,:,2].flatten().std() if type(x) == np.ndarray else x)
all_imgs['channel_B_pixel_min'] = all_imgs['image'].map(lambda x: x[:,:,0].flatten().min() if type(x) == np.ndarray else x)
all_imgs['channel_G_pixel_min'] = all_imgs['image'].map(lambda x: x[:,:,1].flatten().min() if type(x) == np.ndarray else x)
all_imgs['channel_R_pixel_min'] = all_imgs['image'].map(lambda x: x[:,:,2].flatten().min() if type(x) == np.ndarray else x)
all_imgs['channel_B_pixel_max'] = all_imgs['image'].map(lambda x: x[:,:,0].flatten().max() if type(x) == np.ndarray else x)
all_imgs['channel_G_pixel_max'] = all_imgs['image'].map(lambda x: x[:,:,1].flatten().max() if type(x) == np.ndarray else x)
all_imgs['channel_R_pixel_max'] = all_imgs['image'].map(lambda x: x[:,:,2].flatten().max() if type(x) == np.ndarray else x)

Write to disk.

In [34]:
for ind, row in tqdm(all_imgs.iterrows(), total=all_imgs.shape[0]):
    if type(row['image']) == np.ndarray:
        cv2.imwrite(f'./data/loaded/{ind}.jpeg', row['image'])

  0%|          | 0/16767 [00:00<?, ?it/s]

100%|██████████| 16767/16767 [02:36<00:00, 106.82it/s]


In [35]:
all_imgs['image_path'] = all_imgs.apply(lambda x: f'./data/loaded/{x.name}.jpeg' if type(x['image']) == np.ndarray else np.nan, axis=1)

In [36]:
all_imgs.drop('image', axis=1).to_csv('./data/loaded/metadata.csv')

Write to GCS

In [40]:
upload_many_blobs_with_transfer_manager('ai-yoga-trainer-data', filenames=[filename.split('./data/loaded/')[1] for filename in glob('./data/loaded/*')], destination_name_pfx='loaded/', source_directory='./data/loaded/')

Uploaded 2839.jpeg to ai-yoga-trainer-data.
Uploaded 1716.jpeg to ai-yoga-trainer-data.
Uploaded 11205.jpeg to ai-yoga-trainer-data.
Uploaded 9326.jpeg to ai-yoga-trainer-data.
Uploaded 13238.jpeg to ai-yoga-trainer-data.
Uploaded 14007.jpeg to ai-yoga-trainer-data.
Uploaded 723.jpeg to ai-yoga-trainer-data.
Uploaded 4514.jpeg to ai-yoga-trainer-data.
Uploaded 6529.jpeg to ai-yoga-trainer-data.
Uploaded 1346.jpeg to ai-yoga-trainer-data.
Uploaded 6483.jpeg to ai-yoga-trainer-data.
Uploaded 8864.jpeg to ai-yoga-trainer-data.
Uploaded 10947.jpeg to ai-yoga-trainer-data.
Uploaded 689.jpeg to ai-yoga-trainer-data.
Uploaded 11655.jpeg to ai-yoga-trainer-data.
Uploaded 13668.jpeg to ai-yoga-trainer-data.
Uploaded 9776.jpeg to ai-yoga-trainer-data.
Uploaded 16190.jpeg to ai-yoga-trainer-data.
Uploaded 373.jpeg to ai-yoga-trainer-data.
Uploaded 14457.jpeg to ai-yoga-trainer-data.
Uploaded 13392.jpeg to ai-yoga-trainer-data.
Uploaded 2993.jpeg to ai-yoga-trainer-data.
Uploaded 4144.jpeg to ai-y